## Introduction to Cargo Movements endpoint

This notebook will give a tutorial on how to query our cargo movements data through pythonSDK.

In this notebook, we will query data for <b>China Crude Imports</b> as our example.

## Import Libraries

In [1]:
import pandas as pd
import vortexasdk as v
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

c:\Users\OuiWeinJien\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Extract Product & Location ids

Since we are looking for China Crude imports, let's search for those products which contains "Crude" in its name and save it as a variable.

In [3]:
v.Products().search('Crude').to_df(columns = 'all')

2024-09-30 15:44:04,520 vortexasdk.client — WARNING — You are using vortexasdk version 0.73.0, however version 0.74.1 is available.
You should consider upgrading via the 'pip install vortexasdk --upgrade' command.


,id,name,layer.0,leaf,parent.0.name,parent.0.layer.0,parent.0.id,meta.sulphur_min,meta.sulphur_max,meta.api_min,...,hierarchy.0.label,hierarchy.1.id,hierarchy.1.layer,hierarchy.1.label,hierarchy.2.id,hierarchy.2.layer,hierarchy.2.label,hierarchy.3.id,hierarchy.3.layer,hierarchy.3.label
0,1dfa78c965d56ab9d7b953e424d9160e9f79d0275e460c...,Kaliningrad Light Crude,grade,True,Light-Sweet,category,66a98cd9df06660555000f8d4de9bc075e651bce3b57a4...,0.19,0.19,40.0,...,Light-Sweet,6f11b0724c9a4e85ffa7f1445bc768f054af755a090118...,group_product,Crude,54af755a090118dcf9b0724c9a4e9f14745c26165385ff...,group,Crude/Condensates,1dfa78c965d56ab9d7b953e424d9160e9f79d0275e460c...,grade,Kaliningrad Light Crude
1,257e8c09b75a37909af0a200939e95ef9183b4fb5e2a27...,Crude Coconut Oil,grade,True,Biodiesel Feedstock,category,9d52ede1cff0421a8cd7283b0171afe8d23f519dca5f4e...,,,,...,Biodiesel Feedstock,b68cbb746f8b9098c50e2ba36bcad83001a53bd362e903...,group,Clean Petroleum Products,a75fcc09bfc7d16496de3336551bc52b5891838bb7c223...,group_product,Biodiesel,257e8c09b75a37909af0a200939e95ef9183b4fb5e2a27...,grade,Crude Coconut Oil
2,4ff810b966104a658d22e7155596c11bedcd1e3e81594e...,Yuri Korchagin Crude,grade,True,Light-Sweet,category,66a98cd9df06660555000f8d4de9bc075e651bce3b57a4...,0.13,0.13,,...,Crude,66a98cd9df06660555000f8d4de9bc075e651bce3b57a4...,category,Light-Sweet,54af755a090118dcf9b0724c9a4e9f14745c26165385ff...,group,Crude/Condensates,4ff810b966104a658d22e7155596c11bedcd1e3e81594e...,grade,Yuri Korchagin Crude
3,54af755a090118dcf9b0724c9a4e9f14745c26165385ff...,Crude/Condensates,group,False,,,,,,33.395866,...,Crude/Condensates,,,,,,,,,
4,5ccb95de3594474bc74fa5d53e71ae2973977617582538...,Crude Palm Kernel Olein,grade,True,Biodiesel Feedstock,category,9d52ede1cff0421a8cd7283b0171afe8d23f519dca5f4e...,,,,...,Clean Petroleum Products,9d52ede1cff0421a8cd7283b0171afe8d23f519dca5f4e...,category,Biodiesel Feedstock,a75fcc09bfc7d16496de3336551bc52b5891838bb7c223...,group_product,Biodiesel,5ccb95de3594474bc74fa5d53e71ae2973977617582538...,grade,Crude Palm Kernel Olein
5,6f11b0724c9a4e85ffa7f1445bc768f054af755a090118...,Crude,group_product,False,Crude/Condensates,group,54af755a090118dcf9b0724c9a4e9f14745c26165385ff...,,,33.395866,...,Crude/Condensates,6f11b0724c9a4e85ffa7f1445bc768f054af755a090118...,group_product,Crude,,,,,,
6,78610db39212689f5658153d826d0101cc4231a887821a...,Crude Palm Kernel Oil (CPKO),grade,True,Biodiesel Feedstock,category,9d52ede1cff0421a8cd7283b0171afe8d23f519dca5f4e...,,,,...,Biodiesel Feedstock,a75fcc09bfc7d16496de3336551bc52b5891838bb7c223...,group_product,Biodiesel,b68cbb746f8b9098c50e2ba36bcad83001a53bd362e903...,group,Clean Petroleum Products,78610db39212689f5658153d826d0101cc4231a887821a...,grade,Crude Palm Kernel Oil (CPKO)
7,98fa8034b122632b13564878dd75d902faa735b822cac9...,Crude Degummed Soybean Oil,grade,True,Biodiesel Feedstock,category,9d52ede1cff0421a8cd7283b0171afe8d23f519dca5f4e...,,,,...,Biodiesel Feedstock,b68cbb746f8b9098c50e2ba36bcad83001a53bd362e903...,group,Clean Petroleum Products,a75fcc09bfc7d16496de3336551bc52b5891838bb7c223...,group_product,Biodiesel,98fa8034b122632b13564878dd75d902faa735b822cac9...,grade,Crude Degummed Soybean Oil
8,9c536febc4b03c0294b2347f1adc9d23f5b754e3e5b566...,Domestic Sweet Crude (DSW),grade,True,Light-Sweet,category,66a98cd9df06660555000f8d4de9bc075e651bce3b57a4...,0.24,0.24,39.6,...,Crude/Condensates,66a98cd9df06660555000f8d4de9bc075e651bce3b57a4...,category,Light-Sweet,6f11b0724c9a4e85ffa7f1445bc768f054af755a090118...,group_product,Crude,9c536febc4b03c0294b2347f1adc9d23f5b754e3e5b566...,grade,Domestic Sweet Crude (DSW)
9,9fda040ee8844e47b5239051e322d06dd9d2b96f0c3249...,TPAO crude,grade,True,Medium-Sour,category,a7e26956fbb91d786b1d55582981b0d199f72c050958fd...,2.0,2.0,,...,Crude/Condensates,a7e26956fbb91d786b1d55582981b0d199f72c050958fd...,category,Medium-Sour,6f11b0724c9a4e85ffa7f1445bc768f054af755a090118...,group_product,Crude,9fda040ee8844e47b5239051e322d06dd9d2b96f0c3249...,grad

## Assigning id to variables

In [4]:
crude = [p.id for p in v.Products().search('Crude').to_list() if p.name=='Crude/Condensates']

In [5]:
china = [p.id for p in v.Geographies().search('China').to_list() if p.name=='China']
panama_waypoint = [p.id for p in v.Geographies().search('Panama').to_list() if p.name=='Panama Canal']

## Extract Data from CargoMovements endpoint

### 1. Querying the data by specifying parameters

In [6]:
'''
    All cargo movements for crude imports into China between 1st Jan 2024 to now
    
'''
## 1 query
search_result = v.CargoMovements().search(
    filter_activity="unloading_start", #unloading_start - imports, loading_end - exports, storing_state - floating_storage, any_activity for all movements
    filter_time_min=datetime(2024, 1, 1),
    filter_time_max=datetime.today(),
    filter_products = crude,
    filter_destinations = china,
    cm_unit = 'b',
    intra_movements = 'exclude_intra_country'
)

### 2. Selecting desired output columns / Choosing all possible output columns

In [7]:
'''
    Convert data into dataframe.
    Specify columns to be output column.
    Otherwise, type columns = 'all' to output all possible columns.
    
'''
# Choosing desired columns here for demo purpose
cols = [
    'events.cargo_port_load_event.0.location.port.label', # Loading port
    'events.cargo_port_unload_event.0.location.port.label', # Discharging port
    'product.group.label', # Product
    'product.group_product.label', # Product Group
    'product.category.label', # Product Category
    'product.grade.label', # Product Grade
    'quantity', # Quantity
    'status', # Status
    'vessels.0.name', # Vessel Name
    'vessels.0.imo',
    'events.cargo_port_load_event.0.start_timestamp', # Loading Start time
    'events.cargo_port_load_event.0.end_timestamp', # Loading End time (Departure Date)
    'events.cargo_port_unload_event.0.start_timestamp', # Unloading Start time (Arrival Date)
    'events.cargo_port_unload_event.0.end_timestamp', # Unloading end time
    'vessels.0.corporate_entities.charterer.label', # Charterer name
    'events.cargo_sts_event.0.from_vessel_name',
    'events.cargo_sts_event.0.to_vessel_name',
    'events.cargo_sts_event.0.start_timestamp',
    'events.cargo_sts_event.0.end_timestamp',
    'events.cargo_sts_event.0.location.country.label',
    'events.cargo_sts_event.0.location.sts_zone.label',
    ]


# Choosing all columns

# cols = 'all'

df = search_result.to_df(columns = cols)

In [8]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3548 entries, 0 to 3547
Data columns (total 21 columns):
 #   Column                                                Non-Null Count  Dtype              
---  ------                                                --------------  -----              
 0   events.cargo_port_load_event.0.location.port.label    3548 non-null   object             
 1   events.cargo_port_unload_event.0.location.port.label  3548 non-null   object             
 2   product.group.label                                   3548 non-null   object             
 3   product.group_product.label                           3548 non-null   object             
 4   product.category.label                                3548 non-null   object             
 5   product.grade.label                                   3548 non-null   object             
 6   quantity                                              3548 non-null   int64              
 7   status                           

,events.cargo_port_load_event.0.location.port.label,events.cargo_port_unload_event.0.location.port.label,product.group.label,product.group_product.label,product.category.label,product.grade.label,quantity,status,vessels.0.name,vessels.0.imo,...,events.cargo_port_load_event.0.end_timestamp,events.cargo_port_unload_event.0.start_timestamp,events.cargo_port_unload_event.0.end_timestamp,vessels.0.corporate_entities.charterer.label,events.cargo_sts_event.0.from_vessel_name,events.cargo_sts_event.0.to_vessel_name,events.cargo_sts_event.0.start_timestamp,events.cargo_sts_event.0.end_timestamp,events.cargo_sts_event.0.location.country.label,events.cargo_sts_event.0.location.sts_zone.label
0,Kozmino [RU],Dongjiakou [CN],Crude/Condensates,Crude,Medium-Sour,ESPO blend,737108,unloaded_state,CRIUS,9251274,...,2024-03-12 07:40:05+00:00,2024-03-18 05:38:23+00:00,2024-03-20 00:55:51+00:00,,,,NaT,NaT,,
1,Ras Tanura [SA],Quanzhou [CN],Crude/Condensates,Crude,Light-Sour,Arab Extra Light,1123921,unloaded_state,DELTA AMAZON,9748916,...,2024-04-23 22:56:24+00:00,2024-05-15 09:10:02+00:00,2024-05-18 09:10:02+00:00,UNIPEC,,,NaT,NaT,,
2,Juaymah Terminal [SA],Huizhou [CN],Crude/Condensates,Crude,Medium-Sour,Arab Medium,117550,unloaded_state,DELTA APOLLONIA,9516935,...,2024-03-19 04:38:04+00:00,2024-04-11 02:43:06+00:00,2024-04-13 03:25:26+00:00,UNIPEC,,,NaT,NaT,,
3,Kozmino [RU],Dalian [CN],Crude/Condensates,Crude,Medium-Sour,ESPO blend,760969,unloaded_state,EMILY S,9321847,...,2024-01-28 09:20:41+00:00,2024-02-02 04:34:19+00:00,2024-02-03 16:11:36+00:00,,,,NaT,NaT,,
4,Mina Al Ahmadi [KW],Ningbo (Beilun) [CN],Crude/Condensates,Crude,Medium-Sour,Kuwait Export,1481940,unloaded_state,AL KOUT,9653434,...,2024-06-14 19:16:34+00:00,2024-07-06 10:43:23+00:00,2024-07-09 01:57:56+00:00,,,,NaT,NaT,,


## Conclusion

This notebook illustrates the process of querying cargo movement data through the Cargo Movements endpoint. It begins by retrieving product and location IDs from reference data endpoints like Vessels and Geographies. Once the appropriate parameters are set, the API is called, and the desired columns are selected to generate the required table.
